In [2]:
import pandas as pd
import requests

In [190]:
NHL_BASE_URL = "http://statsapi.web.nhl.com"


"""
PLAY-OFF GAMES CODES
==============================================================================
2015 | 03 | 04 | 12 

2015 = season code, first year of the season (e.g., 2015 is for the 2015-16 seasons)
03   = game type code; 1 = preseason, 2 = regular season; 3 = playoffs
04   = playoff only: round number (1st round = 1, 2nd round = 2, ECF/WCF = 3, SCF = 4)
1    = series number: 1-8 in round 1, 1-4 in round 2, 1-2 in round 3, 1 in round 4
2    = game number: 1-7 for any given series
==============================================================================

REGULAR SEASON / PRE SEASON GAMES CODES
==============================================================================
2015 | 02 | 0807
2015 = season code, first year of the season (e.g., 2015 is for the 2015-16 seasons)
02   = game type code; 1 = preseason, 2 = regular season; 3 = playoffs
0807 = game ID; generally 1-1230 in a normal regular season, but sometimes games will be missing 
       (e.g., games cancelled due to weather) and sometimes games will be added on the end, starting
        with 1231 (e.g., make-up games for weather-cancelled games). Numbers are usually approx. 
        1-130ish in the pre-season, but it can be arbitrary.
==============================================================================
"""
years = range(2004, 2005)
game_types = [2]
game_id_preseason = range(1, 140)
game_id_season = range(0, 1300)
cap_break = 1500*18

games_code_list = []
for year in years:
    for game_type in game_types:
        if game_type == 1:
            for game_id in game_id_preseason:
                games_code_list.append("{0}{1:02d}{2:04d}".format(year, game_type, game_id))
        elif game_type == 2:
            for game_id in game_id_season:
                games_code_list.append("{0}{1:02d}{2:04d}".format(year, game_type, game_id))
        elif game_type == 3:
            # 2015 | 03 | 04 | 12
            for playoff in range(1, 5):
                for series in range(1, 9):
                    for game in range(1, 8):
                        games_code_list.append("{0}{1:02d}{2:02d}{3}{4}".format(year, game_type, playoff, series, game))
#print(games_code_list)

game_list = []
error_count = 0

for idx, game_code in enumerate(games_code_list):
    nhl_game_url = "{0}/api/v1/game/{1}/feed/live".format(NHL_BASE_URL, game_code)
    r = requests.get(nhl_game_url)

    print("{0}/{1} - {2}".format(idx, len(games_code_list),game_code))

    if idx > cap_break:
        break

    if r.status_code == 200:    # valid request
        print("Request {0} Successful".format(nhl_game_url))
        r_json = r.json()
        try:
            game = r_json['gameData']
            #start_time = game['datetime']['dateTime']
            #end_time = game['datetime']['endDateTime']

            away_team = game['teams']['away']
            home_team = game['teams']['home']
            away_team_name = away_team['name']
            home_team_name = home_team['name']

            boxscore = r_json['liveData']['boxscore']
            #offical_id = boxscore['officials'][0]['official']['id']
            official_name1 = boxscore['officials'][0]['official']['fullName']
            official_name2 = boxscore['officials'][1]['official']['fullName']

            away_score = boxscore["teams"]["away"]["teamStats"]["teamSkaterStats"]["goals"]
            home_score = boxscore["teams"]["home"]["teamStats"]["teamSkaterStats"]["goals"]
            if away_score > home_score:
                win = away_team_name
            else:
                win = home_team_name

            row_dict = {"code": game_code,
                        'away_team': away_team_name,
                        'away_score' : away_score,
                        'home_team': home_team_name,
                        "home_score" : home_score,
                        "winner" : win,
                        'official_1': official_name1,
                        "official_2" : official_name2}
            game_list.append(row_dict)
            #print('something')
        except (KeyError, IndexError):
            print("ERROR {0}".format(nhl_game_url))
            error_count += 1
    else:
        print("No Response")
print(game_df)

game_df = pd.DataFrame(game_list)
game_df = game_df.set_index("code")
game_df1 = game_df[["home_team","home_score","away_team","away_score","official_1","official_2","winner"]]

def loser(row):
    if row["winner"] == row["away_team"]:
        return row["home_team"]
    return row["away_team"]
    

game_df1["loser"] = game_df1.apply(loser, axis=1)

game_df1.to_csv("2004_data.csv")

0/1300 - 2004020000
No Response
1/1300 - 2004020001
No Response
2/1300 - 2004020002
No Response
3/1300 - 2004020003
No Response
4/1300 - 2004020004
No Response
5/1300 - 2004020005
No Response
6/1300 - 2004020006
No Response
7/1300 - 2004020007
No Response
8/1300 - 2004020008
No Response
9/1300 - 2004020009
No Response
10/1300 - 2004020010
No Response
11/1300 - 2004020011
No Response
12/1300 - 2004020012
No Response
13/1300 - 2004020013
No Response
14/1300 - 2004020014
No Response
15/1300 - 2004020015
No Response
16/1300 - 2004020016
No Response
17/1300 - 2004020017
No Response
18/1300 - 2004020018
No Response
19/1300 - 2004020019
No Response
20/1300 - 2004020020
No Response
21/1300 - 2004020021
No Response
22/1300 - 2004020022
No Response
23/1300 - 2004020023
No Response
24/1300 - 2004020024
No Response
25/1300 - 2004020025
No Response
26/1300 - 2004020026
No Response
27/1300 - 2004020027
No Response
28/1300 - 2004020028
No Response
29/1300 - 2004020029
No Response
30/1300 - 2004020030

246/1300 - 2004020246
No Response
247/1300 - 2004020247
No Response
248/1300 - 2004020248
No Response
249/1300 - 2004020249
No Response
250/1300 - 2004020250
No Response
251/1300 - 2004020251
No Response
252/1300 - 2004020252
No Response
253/1300 - 2004020253
No Response
254/1300 - 2004020254
No Response
255/1300 - 2004020255
No Response
256/1300 - 2004020256
No Response
257/1300 - 2004020257
No Response
258/1300 - 2004020258
No Response
259/1300 - 2004020259
No Response
260/1300 - 2004020260
No Response
261/1300 - 2004020261
No Response
262/1300 - 2004020262
No Response
263/1300 - 2004020263
No Response
264/1300 - 2004020264
No Response
265/1300 - 2004020265
No Response
266/1300 - 2004020266
No Response
267/1300 - 2004020267
No Response
268/1300 - 2004020268
No Response
269/1300 - 2004020269
No Response
270/1300 - 2004020270
No Response
271/1300 - 2004020271
No Response
272/1300 - 2004020272
No Response
273/1300 - 2004020273
No Response
274/1300 - 2004020274
No Response
275/1300 - 200

487/1300 - 2004020487
No Response
488/1300 - 2004020488
No Response
489/1300 - 2004020489
No Response
490/1300 - 2004020490
No Response
491/1300 - 2004020491
No Response
492/1300 - 2004020492
No Response
493/1300 - 2004020493
No Response
494/1300 - 2004020494
No Response
495/1300 - 2004020495
No Response
496/1300 - 2004020496
No Response
497/1300 - 2004020497
No Response
498/1300 - 2004020498
No Response
499/1300 - 2004020499
No Response
500/1300 - 2004020500
No Response
501/1300 - 2004020501
No Response
502/1300 - 2004020502
No Response
503/1300 - 2004020503
No Response
504/1300 - 2004020504
No Response
505/1300 - 2004020505
No Response
506/1300 - 2004020506
No Response
507/1300 - 2004020507
No Response
508/1300 - 2004020508
No Response
509/1300 - 2004020509
No Response
510/1300 - 2004020510
No Response
511/1300 - 2004020511
No Response
512/1300 - 2004020512
No Response
513/1300 - 2004020513
No Response
514/1300 - 2004020514
No Response
515/1300 - 2004020515
No Response
516/1300 - 200

728/1300 - 2004020728
No Response
729/1300 - 2004020729
No Response
730/1300 - 2004020730
No Response
731/1300 - 2004020731
No Response
732/1300 - 2004020732
No Response
733/1300 - 2004020733
No Response
734/1300 - 2004020734
No Response
735/1300 - 2004020735
No Response
736/1300 - 2004020736
No Response
737/1300 - 2004020737
No Response
738/1300 - 2004020738
No Response
739/1300 - 2004020739
No Response
740/1300 - 2004020740
No Response
741/1300 - 2004020741
No Response
742/1300 - 2004020742
No Response
743/1300 - 2004020743
No Response
744/1300 - 2004020744
No Response
745/1300 - 2004020745
No Response
746/1300 - 2004020746
No Response
747/1300 - 2004020747
No Response
748/1300 - 2004020748
No Response
749/1300 - 2004020749
No Response
750/1300 - 2004020750
No Response
751/1300 - 2004020751
No Response
752/1300 - 2004020752
No Response
753/1300 - 2004020753
No Response
754/1300 - 2004020754
No Response
755/1300 - 2004020755
No Response
756/1300 - 2004020756
No Response
757/1300 - 200

969/1300 - 2004020969
No Response
970/1300 - 2004020970
No Response
971/1300 - 2004020971
No Response
972/1300 - 2004020972
No Response
973/1300 - 2004020973
No Response
974/1300 - 2004020974
No Response
975/1300 - 2004020975
No Response
976/1300 - 2004020976
No Response
977/1300 - 2004020977
No Response
978/1300 - 2004020978
No Response
979/1300 - 2004020979
No Response
980/1300 - 2004020980
No Response
981/1300 - 2004020981
No Response
982/1300 - 2004020982
No Response
983/1300 - 2004020983
No Response
984/1300 - 2004020984
No Response
985/1300 - 2004020985
No Response
986/1300 - 2004020986
No Response
987/1300 - 2004020987
No Response
988/1300 - 2004020988
No Response
989/1300 - 2004020989
No Response
990/1300 - 2004020990
No Response
991/1300 - 2004020991
No Response
992/1300 - 2004020992
No Response
993/1300 - 2004020993
No Response
994/1300 - 2004020994
No Response
995/1300 - 2004020995
No Response
996/1300 - 2004020996
No Response
997/1300 - 2004020997
No Response
998/1300 - 200

1204/1300 - 2004021204
No Response
1205/1300 - 2004021205
No Response
1206/1300 - 2004021206
No Response
1207/1300 - 2004021207
No Response
1208/1300 - 2004021208
No Response
1209/1300 - 2004021209
No Response
1210/1300 - 2004021210
No Response
1211/1300 - 2004021211
No Response
1212/1300 - 2004021212
No Response
1213/1300 - 2004021213
No Response
1214/1300 - 2004021214
No Response
1215/1300 - 2004021215
No Response
1216/1300 - 2004021216
No Response
1217/1300 - 2004021217
No Response
1218/1300 - 2004021218
No Response
1219/1300 - 2004021219
No Response
1220/1300 - 2004021220
No Response
1221/1300 - 2004021221
No Response
1222/1300 - 2004021222
No Response
1223/1300 - 2004021223
No Response
1224/1300 - 2004021224
No Response
1225/1300 - 2004021225
No Response
1226/1300 - 2004021226
No Response
1227/1300 - 2004021227
No Response
1228/1300 - 2004021228
No Response
1229/1300 - 2004021229
No Response
1230/1300 - 2004021230
No Response
1231/1300 - 2004021231
No Response
1232/1300 - 20040212

NameError: name 'game_df' is not defined

In [ ]:
df = pd.read_csv("2004_data.csv")
team = df.home_team.unique()
team1 = df.away_team.unique()
official1 = df.official_1.unique()
official2 = df.official_2.unique()
df1 = pd.DataFrame(team)
df2 = pd.DataFrame(team1)
df3 = pd.DataFrame(official1)
df4 = pd.DataFrame(official2)
teams = df1
refs = df3.append(df4)
refs1 = refs[0].unique()
officials = pd.DataFrame(refs1)
team = pd.DataFrame(sorted(teams[0]))
official = pd.DataFrame(sorted(officials[0]))

e = []

for a in df.index:
    for b in sorted(officials[0]):
        for c in sorted(teams[0]):
            if df.loc[a]["away_team"] == c:
                if df.loc[a]["official_1"] == b:
                    d = df.loc[a]["away_score"]
                    e.append({"Ref": b, "Team": c, "Score": d})
                if df.loc[a]["official_2"] == b:
                    d = df.loc[a]["away_score"]
                    e.append({"Ref": b, "Team": c, "Score": d})
            if df.loc[a]["home_team"] == c:
                if df.loc[a]["official_1"] == b:
                    d = df.loc[a]["home_score"]
                    e.append({"Ref": b, "Team": c, "Score": d})
                if df.loc[a]["official_2"] == b:
                    d = df.loc[a]["home_score"]
                    e.append({"Ref": b, "Team": c, "Score": d})
cleaned = pd.DataFrame(e)       
cleaned.to_csv("2004_data_cleaned.csv")

In [168]:
df = pd.read_csv("2015_data.csv")
team = df.home_team.unique()
team1 = df.away_team.unique()
official1 = df.official_1.unique()
official2 = df.official_2.unique()
df1 = pd.DataFrame(team)
df2 = pd.DataFrame(team1)
df3 = pd.DataFrame(official1)
df4 = pd.DataFrame(official2)
teams = df1
refs = df3.append(df4)
refs1 = refs[0].unique()
officials = pd.DataFrame(refs1)
team = pd.DataFrame(sorted(teams[0]))
official = pd.DataFrame(sorted(officials[0]))

e = []

for a in df.index:
    for b in sorted(officials[0]):
        for c in sorted(teams[0]):
            if df.loc[a]["away_team"] == c:
                if df.loc[a]["official_1"] == b:
                    d = df.loc[a]["away_score"]
                    e.append({"Ref": b, "Team": c, "Score": d})
                if df.loc[a]["official_2"] == b:
                    d = df.loc[a]["away_score"]
                    e.append({"Ref": b, "Team": c, "Score": d})
            if df.loc[a]["home_team"] == c:
                if df.loc[a]["official_1"] == b:
                    d = df.loc[a]["home_score"]
                    e.append({"Ref": b, "Team": c, "Score": d})
                if df.loc[a]["official_2"] == b:
                    d = df.loc[a]["home_score"]
                    e.append({"Ref": b, "Team": c, "Score": d})
cleaned = pd.DataFrame(e)       
cleaned.to_csv("2015_data_cleaned.csv")

In [169]:
df = pd.read_csv("2014_data.csv")
team = df.home_team.unique()
team1 = df.away_team.unique()
official1 = df.official_1.unique()
official2 = df.official_2.unique()
df1 = pd.DataFrame(team)
df2 = pd.DataFrame(team1)
df3 = pd.DataFrame(official1)
df4 = pd.DataFrame(official2)
teams = df1
refs = df3.append(df4)
refs1 = refs[0].unique()
officials = pd.DataFrame(refs1)
team = pd.DataFrame(sorted(teams[0]))
official = pd.DataFrame(sorted(officials[0]))

e = []

for a in df.index:
    for b in sorted(officials[0]):
        for c in sorted(teams[0]):
            if df.loc[a]["away_team"] == c:
                if df.loc[a]["official_1"] == b:
                    d = df.loc[a]["away_score"]
                    e.append({"Ref": b, "Team": c, "Score": d})
                if df.loc[a]["official_2"] == b:
                    d = df.loc[a]["away_score"]
                    e.append({"Ref": b, "Team": c, "Score": d})
            if df.loc[a]["home_team"] == c:
                if df.loc[a]["official_1"] == b:
                    d = df.loc[a]["home_score"]
                    e.append({"Ref": b, "Team": c, "Score": d})
                if df.loc[a]["official_2"] == b:
                    d = df.loc[a]["home_score"]
                    e.append({"Ref": b, "Team": c, "Score": d})
cleaned = pd.DataFrame(e)       
cleaned.to_csv("2014_data_cleaned.csv")

In [170]:
df = pd.read_csv("2013_data.csv")
team = df.home_team.unique()
team1 = df.away_team.unique()
official1 = df.official_1.unique()
official2 = df.official_2.unique()
df1 = pd.DataFrame(team)
df2 = pd.DataFrame(team1)
df3 = pd.DataFrame(official1)
df4 = pd.DataFrame(official2)
teams = df1
refs = df3.append(df4)
refs1 = refs[0].unique()
officials = pd.DataFrame(refs1)
team = pd.DataFrame(sorted(teams[0]))
official = pd.DataFrame(sorted(officials[0]))

e = []

for a in df.index:
    for b in sorted(officials[0]):
        for c in sorted(teams[0]):
            if df.loc[a]["away_team"] == c:
                if df.loc[a]["official_1"] == b:
                    d = df.loc[a]["away_score"]
                    e.append({"Ref": b, "Team": c, "Score": d})
                if df.loc[a]["official_2"] == b:
                    d = df.loc[a]["away_score"]
                    e.append({"Ref": b, "Team": c, "Score": d})
            if df.loc[a]["home_team"] == c:
                if df.loc[a]["official_1"] == b:
                    d = df.loc[a]["home_score"]
                    e.append({"Ref": b, "Team": c, "Score": d})
                if df.loc[a]["official_2"] == b:
                    d = df.loc[a]["home_score"]
                    e.append({"Ref": b, "Team": c, "Score": d})
cleaned = pd.DataFrame(e)       
cleaned.to_csv("2013_data_cleaned.csv")

In [171]:
df = pd.read_csv("2012_data.csv")
team = df.home_team.unique()
team1 = df.away_team.unique()
official1 = df.official_1.unique()
official2 = df.official_2.unique()
df1 = pd.DataFrame(team)
df2 = pd.DataFrame(team1)
df3 = pd.DataFrame(official1)
df4 = pd.DataFrame(official2)
teams = df1
refs = df3.append(df4)
refs1 = refs[0].unique()
officials = pd.DataFrame(refs1)
team = pd.DataFrame(sorted(teams[0]))
official = pd.DataFrame(sorted(officials[0]))

e = []

for a in df.index:
    for b in sorted(officials[0]):
        for c in sorted(teams[0]):
            if df.loc[a]["away_team"] == c:
                if df.loc[a]["official_1"] == b:
                    d = df.loc[a]["away_score"]
                    e.append({"Ref": b, "Team": c, "Score": d})
                if df.loc[a]["official_2"] == b:
                    d = df.loc[a]["away_score"]
                    e.append({"Ref": b, "Team": c, "Score": d})
            if df.loc[a]["home_team"] == c:
                if df.loc[a]["official_1"] == b:
                    d = df.loc[a]["home_score"]
                    e.append({"Ref": b, "Team": c, "Score": d})
                if df.loc[a]["official_2"] == b:
                    d = df.loc[a]["home_score"]
                    e.append({"Ref": b, "Team": c, "Score": d})
cleaned = pd.DataFrame(e)       
cleaned.to_csv("2012_data_cleaned.csv")

In [172]:
df = pd.read_csv("2011_data.csv")
team = df.home_team.unique()
team1 = df.away_team.unique()
official1 = df.official_1.unique()
official2 = df.official_2.unique()
df1 = pd.DataFrame(team)
df2 = pd.DataFrame(team1)
df3 = pd.DataFrame(official1)
df4 = pd.DataFrame(official2)
teams = df1
refs = df3.append(df4)
refs1 = refs[0].unique()
officials = pd.DataFrame(refs1)
team = pd.DataFrame(sorted(teams[0]))
official = pd.DataFrame(sorted(officials[0]))

e = []

for a in df.index:
    for b in sorted(officials[0]):
        for c in sorted(teams[0]):
            if df.loc[a]["away_team"] == c:
                if df.loc[a]["official_1"] == b:
                    d = df.loc[a]["away_score"]
                    e.append({"Ref": b, "Team": c, "Score": d})
                if df.loc[a]["official_2"] == b:
                    d = df.loc[a]["away_score"]
                    e.append({"Ref": b, "Team": c, "Score": d})
            if df.loc[a]["home_team"] == c:
                if df.loc[a]["official_1"] == b:
                    d = df.loc[a]["home_score"]
                    e.append({"Ref": b, "Team": c, "Score": d})
                if df.loc[a]["official_2"] == b:
                    d = df.loc[a]["home_score"]
                    e.append({"Ref": b, "Team": c, "Score": d})
cleaned = pd.DataFrame(e)       
cleaned.to_csv("2011_data_cleaned.csv")

In [173]:
df = pd.read_csv("2010_data.csv")
team = df.home_team.unique()
team1 = df.away_team.unique()
official1 = df.official_1.unique()
official2 = df.official_2.unique()
df1 = pd.DataFrame(team)
df2 = pd.DataFrame(team1)
df3 = pd.DataFrame(official1)
df4 = pd.DataFrame(official2)
teams = df1
refs = df3.append(df4)
refs1 = refs[0].unique()
officials = pd.DataFrame(refs1)
team = pd.DataFrame(sorted(teams[0]))
official = pd.DataFrame(sorted(officials[0]))

e = []

for a in df.index:
    for b in sorted(officials[0]):
        for c in sorted(teams[0]):
            if df.loc[a]["away_team"] == c:
                if df.loc[a]["official_1"] == b:
                    d = df.loc[a]["away_score"]
                    e.append({"Ref": b, "Team": c, "Score": d})
                if df.loc[a]["official_2"] == b:
                    d = df.loc[a]["away_score"]
                    e.append({"Ref": b, "Team": c, "Score": d})
            if df.loc[a]["home_team"] == c:
                if df.loc[a]["official_1"] == b:
                    d = df.loc[a]["home_score"]
                    e.append({"Ref": b, "Team": c, "Score": d})
                if df.loc[a]["official_2"] == b:
                    d = df.loc[a]["home_score"]
                    e.append({"Ref": b, "Team": c, "Score": d})
cleaned = pd.DataFrame(e)       
cleaned.to_csv("2010_data_cleaned.csv")

In [174]:
df = pd.read_csv("2009_data.csv")
team = df.home_team.unique()
team1 = df.away_team.unique()
official1 = df.official_1.unique()
official2 = df.official_2.unique()
df1 = pd.DataFrame(team)
df2 = pd.DataFrame(team1)
df3 = pd.DataFrame(official1)
df4 = pd.DataFrame(official2)
teams = df1
refs = df3.append(df4)
refs1 = refs[0].unique()
officials = pd.DataFrame(refs1)
team = pd.DataFrame(sorted(teams[0]))
official = pd.DataFrame(sorted(officials[0]))

e = []

for a in df.index:
    for b in sorted(officials[0]):
        for c in sorted(teams[0]):
            if df.loc[a]["away_team"] == c:
                if df.loc[a]["official_1"] == b:
                    d = df.loc[a]["away_score"]
                    e.append({"Ref": b, "Team": c, "Score": d})
                if df.loc[a]["official_2"] == b:
                    d = df.loc[a]["away_score"]
                    e.append({"Ref": b, "Team": c, "Score": d})
            if df.loc[a]["home_team"] == c:
                if df.loc[a]["official_1"] == b:
                    d = df.loc[a]["home_score"]
                    e.append({"Ref": b, "Team": c, "Score": d})
                if df.loc[a]["official_2"] == b:
                    d = df.loc[a]["home_score"]
                    e.append({"Ref": b, "Team": c, "Score": d})
cleaned = pd.DataFrame(e)       
cleaned.to_csv("2009_data_cleaned.csv")

In [175]:
df = pd.read_csv("2008_data.csv")
team = df.home_team.unique()
team1 = df.away_team.unique()
official1 = df.official_1.unique()
official2 = df.official_2.unique()
df1 = pd.DataFrame(team)
df2 = pd.DataFrame(team1)
df3 = pd.DataFrame(official1)
df4 = pd.DataFrame(official2)
teams = df1
refs = df3.append(df4)
refs1 = refs[0].unique()
officials = pd.DataFrame(refs1)
team = pd.DataFrame(sorted(teams[0]))
official = pd.DataFrame(sorted(officials[0]))

e = []

for a in df.index:
    for b in sorted(officials[0]):
        for c in sorted(teams[0]):
            if df.loc[a]["away_team"] == c:
                if df.loc[a]["official_1"] == b:
                    d = df.loc[a]["away_score"]
                    e.append({"Ref": b, "Team": c, "Score": d})
                if df.loc[a]["official_2"] == b:
                    d = df.loc[a]["away_score"]
                    e.append({"Ref": b, "Team": c, "Score": d})
            if df.loc[a]["home_team"] == c:
                if df.loc[a]["official_1"] == b:
                    d = df.loc[a]["home_score"]
                    e.append({"Ref": b, "Team": c, "Score": d})
                if df.loc[a]["official_2"] == b:
                    d = df.loc[a]["home_score"]
                    e.append({"Ref": b, "Team": c, "Score": d})
cleaned = pd.DataFrame(e)       
cleaned.to_csv("2008_data_cleaned.csv")

In [176]:
df = pd.read_csv("2007_data.csv")
team = df.home_team.unique()
team1 = df.away_team.unique()
official1 = df.official_1.unique()
official2 = df.official_2.unique()
df1 = pd.DataFrame(team)
df2 = pd.DataFrame(team1)
df3 = pd.DataFrame(official1)
df4 = pd.DataFrame(official2)
teams = df1
refs = df3.append(df4)
refs1 = refs[0].unique()
officials = pd.DataFrame(refs1)
team = pd.DataFrame(sorted(teams[0]))
official = pd.DataFrame(sorted(officials[0]))

e = []

for a in df.index:
    for b in sorted(officials[0]):
        for c in sorted(teams[0]):
            if df.loc[a]["away_team"] == c:
                if df.loc[a]["official_1"] == b:
                    d = df.loc[a]["away_score"]
                    e.append({"Ref": b, "Team": c, "Score": d})
                if df.loc[a]["official_2"] == b:
                    d = df.loc[a]["away_score"]
                    e.append({"Ref": b, "Team": c, "Score": d})
            if df.loc[a]["home_team"] == c:
                if df.loc[a]["official_1"] == b:
                    d = df.loc[a]["home_score"]
                    e.append({"Ref": b, "Team": c, "Score": d})
                if df.loc[a]["official_2"] == b:
                    d = df.loc[a]["home_score"]
                    e.append({"Ref": b, "Team": c, "Score": d})
cleaned = pd.DataFrame(e)       
cleaned.to_csv("2007_data_cleaned.csv")

In [177]:
df = pd.read_csv("2006_data.csv")
team = df.home_team.unique()
team1 = df.away_team.unique()
official1 = df.official_1.unique()
official2 = df.official_2.unique()
df1 = pd.DataFrame(team)
df2 = pd.DataFrame(team1)
df3 = pd.DataFrame(official1)
df4 = pd.DataFrame(official2)
teams = df1
refs = df3.append(df4)
refs1 = refs[0].unique()
officials = pd.DataFrame(refs1)
team = pd.DataFrame(sorted(teams[0]))
official = pd.DataFrame(sorted(officials[0]))

e = []

for a in df.index:
    for b in sorted(officials[0]):
        for c in sorted(teams[0]):
            if df.loc[a]["away_team"] == c:
                if df.loc[a]["official_1"] == b:
                    d = df.loc[a]["away_score"]
                    e.append({"Ref": b, "Team": c, "Score": d})
                if df.loc[a]["official_2"] == b:
                    d = df.loc[a]["away_score"]
                    e.append({"Ref": b, "Team": c, "Score": d})
            if df.loc[a]["home_team"] == c:
                if df.loc[a]["official_1"] == b:
                    d = df.loc[a]["home_score"]
                    e.append({"Ref": b, "Team": c, "Score": d})
                if df.loc[a]["official_2"] == b:
                    d = df.loc[a]["home_score"]
                    e.append({"Ref": b, "Team": c, "Score": d})
cleaned = pd.DataFrame(e)       
cleaned.to_csv("2006_data_cleaned.csv")

In [178]:
df = pd.read_csv("2005_data.csv")
team = df.home_team.unique()
team1 = df.away_team.unique()
official1 = df.official_1.unique()
official2 = df.official_2.unique()
df1 = pd.DataFrame(team)
df2 = pd.DataFrame(team1)
df3 = pd.DataFrame(official1)
df4 = pd.DataFrame(official2)
teams = df1
refs = df3.append(df4)
refs1 = refs[0].unique()
officials = pd.DataFrame(refs1)
team = pd.DataFrame(sorted(teams[0]))
official = pd.DataFrame(sorted(officials[0]))

e = []

for a in df.index:
    for b in sorted(officials[0]):
        for c in sorted(teams[0]):
            if df.loc[a]["away_team"] == c:
                if df.loc[a]["official_1"] == b:
                    d = df.loc[a]["away_score"]
                    e.append({"Ref": b, "Team": c, "Score": d})
                if df.loc[a]["official_2"] == b:
                    d = df.loc[a]["away_score"]
                    e.append({"Ref": b, "Team": c, "Score": d})
            if df.loc[a]["home_team"] == c:
                if df.loc[a]["official_1"] == b:
                    d = df.loc[a]["home_score"]
                    e.append({"Ref": b, "Team": c, "Score": d})
                if df.loc[a]["official_2"] == b:
                    d = df.loc[a]["home_score"]
                    e.append({"Ref": b, "Team": c, "Score": d})
cleaned = pd.DataFrame(e)       
cleaned.to_csv("2005_data_cleaned.csv")

In [179]:
df = pd.read_csv("2004_data.csv")
team = df.home_team.unique()
team1 = df.away_team.unique()
official1 = df.official_1.unique()
official2 = df.official_2.unique()
df1 = pd.DataFrame(team)
df2 = pd.DataFrame(team1)
df3 = pd.DataFrame(official1)
df4 = pd.DataFrame(official2)
teams = df1
refs = df3.append(df4)
refs1 = refs[0].unique()
officials = pd.DataFrame(refs1)
team = pd.DataFrame(sorted(teams[0]))
official = pd.DataFrame(sorted(officials[0]))

e = []

for a in df.index:
    for b in sorted(officials[0]):
        for c in sorted(teams[0]):
            if df.loc[a]["away_team"] == c:
                if df.loc[a]["official_1"] == b:
                    d = df.loc[a]["away_score"]
                    e.append({"Ref": b, "Team": c, "Score": d})
                if df.loc[a]["official_2"] == b:
                    d = df.loc[a]["away_score"]
                    e.append({"Ref": b, "Team": c, "Score": d})
            if df.loc[a]["home_team"] == c:
                if df.loc[a]["official_1"] == b:
                    d = df.loc[a]["home_score"]
                    e.append({"Ref": b, "Team": c, "Score": d})
                if df.loc[a]["official_2"] == b:
                    d = df.loc[a]["home_score"]
                    e.append({"Ref": b, "Team": c, "Score": d})
cleaned = pd.DataFrame(e)       
cleaned.to_csv("2004_data_cleaned.csv")

FileNotFoundError: File b'2004_data.csv' does not exist

In [180]:
df = pd.read_csv("2003_data.csv")
team = df.home_team.unique()
team1 = df.away_team.unique()
official1 = df.official_1.unique()
official2 = df.official_2.unique()
df1 = pd.DataFrame(team)
df2 = pd.DataFrame(team1)
df3 = pd.DataFrame(official1)
df4 = pd.DataFrame(official2)
teams = df1
refs = df3.append(df4)
refs1 = refs[0].unique()
officials = pd.DataFrame(refs1)
team = pd.DataFrame(sorted(teams[0]))
official = pd.DataFrame(sorted(officials[0]))

e = []

for a in df.index:
    for b in sorted(officials[0]):
        for c in sorted(teams[0]):
            if df.loc[a]["away_team"] == c:
                if df.loc[a]["official_1"] == b:
                    d = df.loc[a]["away_score"]
                    e.append({"Ref": b, "Team": c, "Score": d})
                if df.loc[a]["official_2"] == b:
                    d = df.loc[a]["away_score"]
                    e.append({"Ref": b, "Team": c, "Score": d})
            if df.loc[a]["home_team"] == c:
                if df.loc[a]["official_1"] == b:
                    d = df.loc[a]["home_score"]
                    e.append({"Ref": b, "Team": c, "Score": d})
                if df.loc[a]["official_2"] == b:
                    d = df.loc[a]["home_score"]
                    e.append({"Ref": b, "Team": c, "Score": d})
cleaned = pd.DataFrame(e)       
cleaned.to_csv("2003_data_cleaned.csv")

In [181]:
df = pd.read_csv("2002_data.csv")
team = df.home_team.unique()
team1 = df.away_team.unique()
official1 = df.official_1.unique()
official2 = df.official_2.unique()
df1 = pd.DataFrame(team)
df2 = pd.DataFrame(team1)
df3 = pd.DataFrame(official1)
df4 = pd.DataFrame(official2)
teams = df1
refs = df3.append(df4)
refs1 = refs[0].unique()
officials = pd.DataFrame(refs1)
team = pd.DataFrame(sorted(teams[0]))
official = pd.DataFrame(sorted(officials[0]))

e = []

for a in df.index:
    for b in sorted(officials[0]):
        for c in sorted(teams[0]):
            if df.loc[a]["away_team"] == c:
                if df.loc[a]["official_1"] == b:
                    d = df.loc[a]["away_score"]
                    e.append({"Ref": b, "Team": c, "Score": d})
                if df.loc[a]["official_2"] == b:
                    d = df.loc[a]["away_score"]
                    e.append({"Ref": b, "Team": c, "Score": d})
            if df.loc[a]["home_team"] == c:
                if df.loc[a]["official_1"] == b:
                    d = df.loc[a]["home_score"]
                    e.append({"Ref": b, "Team": c, "Score": d})
                if df.loc[a]["official_2"] == b:
                    d = df.loc[a]["home_score"]
                    e.append({"Ref": b, "Team": c, "Score": d})
cleaned = pd.DataFrame(e)       
cleaned.to_csv("2002_data_cleaned.csv")

In [182]:
df = pd.read_csv("2001_data.csv")
team = df.home_team.unique()
team1 = df.away_team.unique()
official1 = df.official_1.unique()
official2 = df.official_2.unique()
df1 = pd.DataFrame(team)
df2 = pd.DataFrame(team1)
df3 = pd.DataFrame(official1)
df4 = pd.DataFrame(official2)
teams = df1
refs = df3.append(df4)
refs1 = refs[0].unique()
officials = pd.DataFrame(refs1)
team = pd.DataFrame(sorted(teams[0]))
official = pd.DataFrame(sorted(officials[0]))

e = []

for a in df.index:
    for b in sorted(officials[0]):
        for c in sorted(teams[0]):
            if df.loc[a]["away_team"] == c:
                if df.loc[a]["official_1"] == b:
                    d = df.loc[a]["away_score"]
                    e.append({"Ref": b, "Team": c, "Score": d})
                if df.loc[a]["official_2"] == b:
                    d = df.loc[a]["away_score"]
                    e.append({"Ref": b, "Team": c, "Score": d})
            if df.loc[a]["home_team"] == c:
                if df.loc[a]["official_1"] == b:
                    d = df.loc[a]["home_score"]
                    e.append({"Ref": b, "Team": c, "Score": d})
                if df.loc[a]["official_2"] == b:
                    d = df.loc[a]["home_score"]
                    e.append({"Ref": b, "Team": c, "Score": d})
cleaned = pd.DataFrame(e)       
cleaned.to_csv("2001_data_cleaned.csv")

In [183]:
df = pd.read_csv("2000_data.csv")
team = df.home_team.unique()
team1 = df.away_team.unique()
official1 = df.official_1.unique()
official2 = df.official_2.unique()
df1 = pd.DataFrame(team)
df2 = pd.DataFrame(team1)
df3 = pd.DataFrame(official1)
df4 = pd.DataFrame(official2)
teams = df1
refs = df3.append(df4)
refs1 = refs[0].unique()
officials = pd.DataFrame(refs1)
team = pd.DataFrame(sorted(teams[0]))
official = pd.DataFrame(sorted(officials[0]))

e = []

for a in df.index:
    for b in sorted(officials[0]):
        for c in sorted(teams[0]):
            if df.loc[a]["away_team"] == c:
                if df.loc[a]["official_1"] == b:
                    d = df.loc[a]["away_score"]
                    e.append({"Ref": b, "Team": c, "Score": d})
                if df.loc[a]["official_2"] == b:
                    d = df.loc[a]["away_score"]
                    e.append({"Ref": b, "Team": c, "Score": d})
            if df.loc[a]["home_team"] == c:
                if df.loc[a]["official_1"] == b:
                    d = df.loc[a]["home_score"]
                    e.append({"Ref": b, "Team": c, "Score": d})
                if df.loc[a]["official_2"] == b:
                    d = df.loc[a]["home_score"]
                    e.append({"Ref": b, "Team": c, "Score": d})
cleaned = pd.DataFrame(e)       
cleaned.to_csv("2000_data_cleaned.csv")

In [184]:
df = pd.read_csv("1999_data.csv")
team = df.home_team.unique()
team1 = df.away_team.unique()
official1 = df.official_1.unique()
official2 = df.official_2.unique()
df1 = pd.DataFrame(team)
df2 = pd.DataFrame(team1)
df3 = pd.DataFrame(official1)
df4 = pd.DataFrame(official2)
teams = df1
refs = df3.append(df4)
refs1 = refs[0].unique()
officials = pd.DataFrame(refs1)
team = pd.DataFrame(sorted(teams[0]))
official = pd.DataFrame(sorted(officials[0]))

e = []

for a in df.index:
    for b in sorted(officials[0]):
        for c in sorted(teams[0]):
            if df.loc[a]["away_team"] == c:
                if df.loc[a]["official_1"] == b:
                    d = df.loc[a]["away_score"]
                    e.append({"Ref": b, "Team": c, "Score": d})
                if df.loc[a]["official_2"] == b:
                    d = df.loc[a]["away_score"]
                    e.append({"Ref": b, "Team": c, "Score": d})
            if df.loc[a]["home_team"] == c:
                if df.loc[a]["official_1"] == b:
                    d = df.loc[a]["home_score"]
                    e.append({"Ref": b, "Team": c, "Score": d})
                if df.loc[a]["official_2"] == b:
                    d = df.loc[a]["home_score"]
                    e.append({"Ref": b, "Team": c, "Score": d})
cleaned = pd.DataFrame(e)       
cleaned.to_csv("1999_data_cleaned.csv")

In [185]:
df = pd.read_csv("1998_data.csv")
team = df.home_team.unique()
team1 = df.away_team.unique()
official1 = df.official_1.unique()
official2 = df.official_2.unique()
df1 = pd.DataFrame(team)
df2 = pd.DataFrame(team1)
df3 = pd.DataFrame(official1)
df4 = pd.DataFrame(official2)
teams = df1
refs = df3.append(df4)
refs1 = refs[0].unique()
officials = pd.DataFrame(refs1)
team = pd.DataFrame(sorted(teams[0]))
official = pd.DataFrame(sorted(officials[0]))

e = []

for a in df.index:
    for b in sorted(officials[0]):
        for c in sorted(teams[0]):
            if df.loc[a]["away_team"] == c:
                if df.loc[a]["official_1"] == b:
                    d = df.loc[a]["away_score"]
                    e.append({"Ref": b, "Team": c, "Score": d})
                if df.loc[a]["official_2"] == b:
                    d = df.loc[a]["away_score"]
                    e.append({"Ref": b, "Team": c, "Score": d})
            if df.loc[a]["home_team"] == c:
                if df.loc[a]["official_1"] == b:
                    d = df.loc[a]["home_score"]
                    e.append({"Ref": b, "Team": c, "Score": d})
                if df.loc[a]["official_2"] == b:
                    d = df.loc[a]["home_score"]
                    e.append({"Ref": b, "Team": c, "Score": d})
cleaned = pd.DataFrame(e)       
cleaned.to_csv("1998_data_cleaned.csv")

In [186]:
df = pd.read_csv("1997_data.csv")
team = df.home_team.unique()
team1 = df.away_team.unique()
official1 = df.official_1.unique()
official2 = df.official_2.unique()
df1 = pd.DataFrame(team)
df2 = pd.DataFrame(team1)
df3 = pd.DataFrame(official1)
df4 = pd.DataFrame(official2)
teams = df1
refs = df3.append(df4)
refs1 = refs[0].unique()
officials = pd.DataFrame(refs1)
team = pd.DataFrame(sorted(teams[0]))
official = pd.DataFrame(sorted(officials[0]))

e = []

for a in df.index:
    for b in sorted(officials[0]):
        for c in sorted(teams[0]):
            if df.loc[a]["away_team"] == c:
                if df.loc[a]["official_1"] == b:
                    d = df.loc[a]["away_score"]
                    e.append({"Ref": b, "Team": c, "Score": d})
                if df.loc[a]["official_2"] == b:
                    d = df.loc[a]["away_score"]
                    e.append({"Ref": b, "Team": c, "Score": d})
            if df.loc[a]["home_team"] == c:
                if df.loc[a]["official_1"] == b:
                    d = df.loc[a]["home_score"]
                    e.append({"Ref": b, "Team": c, "Score": d})
                if df.loc[a]["official_2"] == b:
                    d = df.loc[a]["home_score"]
                    e.append({"Ref": b, "Team": c, "Score": d})
cleaned = pd.DataFrame(e)       
cleaned.to_csv("1997_data_cleaned.csv")

In [187]:
df = pd.read_csv("1996_data.csv")
team = df.home_team.unique()
team1 = df.away_team.unique()
official1 = df.official_1.unique()
official2 = df.official_2.unique()
df1 = pd.DataFrame(team)
df2 = pd.DataFrame(team1)
df3 = pd.DataFrame(official1)
df4 = pd.DataFrame(official2)
teams = df1
refs = df3.append(df4)
refs1 = refs[0].unique()
officials = pd.DataFrame(refs1)
team = pd.DataFrame(sorted(teams[0]))
official = pd.DataFrame(sorted(officials[0]))

e = []

for a in df.index:
    for b in sorted(officials[0]):
        for c in sorted(teams[0]):
            if df.loc[a]["away_team"] == c:
                if df.loc[a]["official_1"] == b:
                    d = df.loc[a]["away_score"]
                    e.append({"Ref": b, "Team": c, "Score": d})
                if df.loc[a]["official_2"] == b:
                    d = df.loc[a]["away_score"]
                    e.append({"Ref": b, "Team": c, "Score": d})
            if df.loc[a]["home_team"] == c:
                if df.loc[a]["official_1"] == b:
                    d = df.loc[a]["home_score"]
                    e.append({"Ref": b, "Team": c, "Score": d})
                if df.loc[a]["official_2"] == b:
                    d = df.loc[a]["home_score"]
                    e.append({"Ref": b, "Team": c, "Score": d})
cleaned = pd.DataFrame(e)       
cleaned.to_csv("1996_data_cleaned.csv")

In [188]:
df = pd.read_csv("1995_data.csv")
team = df.home_team.unique()
team1 = df.away_team.unique()
official1 = df.official_1.unique()
official2 = df.official_2.unique()
df1 = pd.DataFrame(team)
df2 = pd.DataFrame(team1)
df3 = pd.DataFrame(official1)
df4 = pd.DataFrame(official2)
teams = df1
refs = df3.append(df4)
refs1 = refs[0].unique()
officials = pd.DataFrame(refs1)
team = pd.DataFrame(sorted(teams[0]))
official = pd.DataFrame(sorted(officials[0]))

e = []

for a in df.index:
    for b in sorted(officials[0]):
        for c in sorted(teams[0]):
            if df.loc[a]["away_team"] == c:
                if df.loc[a]["official_1"] == b:
                    d = df.loc[a]["away_score"]
                    e.append({"Ref": b, "Team": c, "Score": d})
                if df.loc[a]["official_2"] == b:
                    d = df.loc[a]["away_score"]
                    e.append({"Ref": b, "Team": c, "Score": d})
            if df.loc[a]["home_team"] == c:
                if df.loc[a]["official_1"] == b:
                    d = df.loc[a]["home_score"]
                    e.append({"Ref": b, "Team": c, "Score": d})
                if df.loc[a]["official_2"] == b:
                    d = df.loc[a]["home_score"]
                    e.append({"Ref": b, "Team": c, "Score": d})
cleaned = pd.DataFrame(e)       
cleaned.to_csv("1995_data_cleaned.csv")

In [189]:
df = pd.read_csv("1994_data.csv")
team = df.home_team.unique()
team1 = df.away_team.unique()
official1 = df.official_1.unique()
official2 = df.official_2.unique()
df1 = pd.DataFrame(team)
df2 = pd.DataFrame(team1)
df3 = pd.DataFrame(official1)
df4 = pd.DataFrame(official2)
teams = df1
refs = df3.append(df4)
refs1 = refs[0].unique()
officials = pd.DataFrame(refs1)
team = pd.DataFrame(sorted(teams[0]))
official = pd.DataFrame(sorted(officials[0]))

e = []

for a in df.index:
    for b in sorted(officials[0]):
        for c in sorted(teams[0]):
            if df.loc[a]["away_team"] == c:
                if df.loc[a]["official_1"] == b:
                    d = df.loc[a]["away_score"]
                    e.append({"Ref": b, "Team": c, "Score": d})
                if df.loc[a]["official_2"] == b:
                    d = df.loc[a]["away_score"]
                    e.append({"Ref": b, "Team": c, "Score": d})
            if df.loc[a]["home_team"] == c:
                if df.loc[a]["official_1"] == b:
                    d = df.loc[a]["home_score"]
                    e.append({"Ref": b, "Team": c, "Score": d})
                if df.loc[a]["official_2"] == b:
                    d = df.loc[a]["home_score"]
                    e.append({"Ref": b, "Team": c, "Score": d})
cleaned = pd.DataFrame(e)       
cleaned.to_csv("1994_data_cleaned.csv")